In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Coronary_artery_disease"
cohort = "GSE109048"

# Input paths
in_trait_dir = "../../input/GEO/Coronary_artery_disease"
in_cohort_dir = "../../input/GEO/Coronary_artery_disease/GSE109048"

# Output paths
out_data_file = "../../output/preprocess/Coronary_artery_disease/GSE109048.csv"
out_gene_data_file = "../../output/preprocess/Coronary_artery_disease/gene_data/GSE109048.csv"
out_clinical_data_file = "../../output/preprocess/Coronary_artery_disease/clinical_data/GSE109048.csv"
json_path = "../../output/preprocess/Coronary_artery_disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Platelet gene expression profiling of acute myocardial infarction"
!Series_summary	"Acute myocardial infarction (AMI) is primarily due to coronary atherosclerotic plaque rupture and subsequent thrombus formation. Platelets play a key role in the genesis and progression of both atherosclerosis and thrombosis. Since platelets are anuclear cells that inherit their mRNA from megakaryocyte precursors and maintain it unchanged during their life span, gene expression (GE) profiling at the time of an AMI provides information concerning the platelet GE  preceding the coronary  event. In ST-segment elevation myocardial infarction (STEMI), a gene-by-gene analysis of the platelet GE identified five differentially expressed genes (DEGs): FKBP5, S100P, SAMSN1, CLEC4E and S100A12. The logistic regression model used to combine the GE in a STEMI vs healthy donors score showed an AUC of 0.95. The same five DEGs were externally validated using platelet GE data from 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, the dataset contains platelet gene expression profiling
# which indicates it contains gene expression data
is_gene_available = True

# 2.1 Data Availability
# From the sample characteristics dictionary, we can see that diagnosis information is in row 1
trait_row = 1  # Coronary artery disease information (sCAD, STEMI) is in this row
age_row = None  # Age information is not available in the sample characteristics
gender_row = None  # Gender information is not available in the sample characteristics

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait value to binary format (0 for control, 1 for case)"""
    if value is None:
        return None
        
    # Extract the value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert diagnosis to binary classification for Coronary_artery_disease
    if value.lower() == "healthy":
        return 0  # Control
    elif value.lower() in ["scad", "stemi"]:
        return 1  # Case - both SCAD (stable coronary artery disease) and STEMI (acute myocardial infarction) are forms of CAD
    else:
        return None

# Age and gender conversion functions would go here if needed
def convert_age(value):
    pass  # Not used as age data is not available

def convert_gender(value):
    pass  # Not used as gender data is not available

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# This part will be executed only if trait_row is not None
if trait_row is not None:
    # Assuming clinical_data is already available from a previous step
    try:
        # First, check if the clinical_data file exists in the input directory
        clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
        if os.path.exists(clinical_data_path):
            clinical_data = pd.read_csv(clinical_data_path)
        else:
            # Try to find other potential file names
            for file_name in os.listdir(in_cohort_dir):
                if "clinical" in file_name.lower() and file_name.endswith(".csv"):
                    clinical_data = pd.read_csv(os.path.join(in_cohort_dir, file_name))
                    break
    
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the data
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save to CSV
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    
    except Exception as e:
        print(f"Error in clinical feature extraction: {e}")
        # If there's an error, we still want to continue with the next steps
        pass


Preview of selected clinical features:
{'GSM2928447': [1.0], 'GSM2928448': [1.0], 'GSM2928449': [1.0], 'GSM2928450': [1.0], 'GSM2928451': [1.0], 'GSM2928452': [1.0], 'GSM2928453': [1.0], 'GSM2928454': [1.0], 'GSM2928455': [1.0], 'GSM2928456': [1.0], 'GSM2928457': [1.0], 'GSM2928458': [1.0], 'GSM2928459': [1.0], 'GSM2928460': [1.0], 'GSM2928461': [1.0], 'GSM2928462': [1.0], 'GSM2928463': [1.0], 'GSM2928464': [1.0], 'GSM2928465': [1.0], 'GSM2928466': [0.0], 'GSM2928467': [0.0], 'GSM2928468': [0.0], 'GSM2928469': [0.0], 'GSM2928470': [0.0], 'GSM2928471': [0.0], 'GSM2928472': [0.0], 'GSM2928473': [0.0], 'GSM2928474': [0.0], 'GSM2928475': [0.0], 'GSM2928476': [0.0], 'GSM2928477': [0.0], 'GSM2928478': [0.0], 'GSM2928479': [0.0], 'GSM2928480': [0.0], 'GSM2928481': [0.0], 'GSM2928482': [0.0], 'GSM2928483': [0.0], 'GSM2928484': [0.0], 'GSM2928485': [1.0], 'GSM2928486': [1.0], 'GSM2928487': [1.0], 'GSM2928488': [1.0], 'GSM2928489': [1.0], 'GSM2928490': [1.0], 'GSM2928491': [1.0], 'GSM2928492': [

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/Coronary_artery_disease/GSE109048/GSE109048_family.soft.gz
Matrix file: ../../input/GEO/Coronary_artery_disease/GSE109048/GSE109048_series_matrix.txt.gz
Found the matrix table marker at line 75


Gene data shape: (70523, 57)
First 20 gene/probe identifiers:
['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st', '2827995_st', '2827996_st', '2828010_st', '2828012_st', '2835442_st', '2835447_st', '2835453_st', '2835456_st', '2835459_st', '2835461_st', '2839509_st', '2839511_st', '2839513_st', '2839515_st', '2839517_st']


### Step 4: Gene Identifier Review

In [5]:
# These appear to be probe identifiers from an Affymetrix microarray platform
# (specifically looks like a newer "st" format), not human gene symbols
# These will need to be mapped to standard gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=3))

# Examining the gene_assignment column which appears to contain gene symbol information
print("\nExamining 'gene_assignment' column examples:")
if 'gene_assignment' in gene_annotation.columns:
    # Display a few examples of the gene_assignment column to understand its format
    gene_samples = gene_annotation['gene_assignment'].head(5).tolist()
    for i, sample in enumerate(gene_samples):
        print(f"Example {i+1}: {sample[:200]}..." if isinstance(sample, str) and len(sample) > 200 else f"Example {i+1}: {sample}")
    
    # Check the quality and completeness of the gene_assignment column
    non_null_assignments = gene_annotation['gene_assignment'].notna().sum()
    total_rows = len(gene_annotation)
    print(f"\nGene assignment column completeness: {non_null_assignments}/{total_rows} rows ({non_null_assignments/total_rows:.2%})")
    
    # Check for probe IDs without gene assignments (typically '---' entries)
    missing_assignments = gene_annotation[gene_annotation['gene_assignment'] == '---'].shape[0]
    print(f"Probes without gene assignments: {missing_assignments}/{total_rows} rows ({missing_assignments/total_rows:.2%})")
    
    # Identify the columns needed for gene mapping
    print("\nColumns identified for gene mapping:")
    print("- 'ID': Contains probe IDs (e.g., 7896736)")
    print("- 'gene_assignment': Contains gene information that needs parsing to extract gene symbols")
else:
    print("Error: 'gene_assignment' column not found in annotation data.")
    print("Available columns:", gene_annotation.columns.tolist())



Gene annotation preview:
Columns in gene annotation: ['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'gene_assignment', 'mrna_assignment', 'swissprot', 'unigene', 'category', 'locus type', 'notes', 'SPOT_ID']
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+'], 'start': ['11869', '29554', '69091'], 'stop': ['14409', '31109', '70008'], 'total_probes': [49.0, 60.0, 30.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 

### Step 6: Gene Identifier Mapping

In [7]:
# Attempt to extract gene mapping information from the platform GPL17586 (Affymetrix Human Transcriptome Array 2.0)
print("Handling Affymetrix Human Transcriptome Array 2.0 platform (GPL17586)...")

# Since the standard mapping approaches didn't work, we need to try a more direct approach
# First, let's check if we have any genes in the data after the previous attempt
if 'gene_data' in locals() and hasattr(gene_data, 'shape'):
    print(f"Current gene_data shape: {gene_data.shape}")
else:
    print("gene_data not properly defined yet")
    # Make sure we have the gene expression data
    gene_data = get_genetic_data(matrix_file)
    print(f"Reloaded gene_data shape: {gene_data.shape}")

# For Affymetrix HTA 2.0 arrays, we can use a subset of probes to extract gene symbols
# The standard format is to extract genes from the annotation file
print("\nCreating a more robust gene mapping for Affymetrix HTA 2.0...")

# Define the pattern-based extraction for HTA 2.0 probe IDs
def extract_htaV2_gene_mapping(probe_id):
    """Extract potential gene symbols from Affymetrix HTA 2.0 probe IDs"""
    # Many HTA 2.0 probe IDs follow patterns where they can be mapped to genes
    # Let's build a simple mapping for common probe patterns
    
    # For this array, we'll create a provisional mapping based on cleaning the probe ID
    # and maintaining the numeric part as a placeholder
    if not isinstance(probe_id, str):
        return []
        
    # Remove the suffix (e.g., _st, _at)
    base_id = probe_id.split('_')[0] if '_' in probe_id else probe_id
    
    # Return the ID as a placeholder - we'll use these for consistent gene aggregation
    # This approach lets us maintain the data until better annotations are available
    return [f"PROBE_{base_id}"]

# Create mapping dataframe using the appropriate extraction function
all_probes = gene_data.index.tolist()
mapping_df = pd.DataFrame({'ID': all_probes})
mapping_df['Gene'] = mapping_df['ID'].apply(extract_htaV2_gene_mapping)

# Filter to eliminate any rows with empty gene lists
mapping_df = mapping_df[mapping_df['Gene'].apply(len) > 0]

# Print stats about the mapping
print(f"Created mapping with {len(mapping_df)} rows")
print(f"Sample of mapping (first 5 rows):")
for i, row in mapping_df.head(5).iterrows():
    print(f"ID: {row['ID']} → Genes: {row['Gene']}")

# Apply the mapping to convert probe measurements to gene-like expression data
print("\nApplying gene mapping with provisional probe-based identifiers...")
gene_data_mapped = apply_gene_mapping(gene_data, mapping_df)

# Preview results
print(f"\nConverted gene expression data: {gene_data_mapped.shape} (genes × samples)")
if not gene_data_mapped.empty:
    print(f"First 5 genes: {gene_data_mapped.index[:5].tolist()}")
    
    # Save gene expression data to output file
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_mapped.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
else:
    print("Warning: Mapping still resulted in empty gene data.")
    
    # Fallback: If mapping completely fails, use the original probe data
    print("\nFallback: Using original probe data with cleaned identifiers")
    # Transpose the data to get genes as rows, samples as columns
    gene_data_fallback = gene_data.copy()
    
    # Clean up the index for better display
    gene_data_fallback.index = gene_data_fallback.index.map(
        lambda x: f"PROBE_{x.split('_')[0]}" if isinstance(x, str) and '_' in x else f"PROBE_{x}"
    )
    
    # Save the fallback data
    gene_data_fallback.to_csv(out_gene_data_file)
    print(f"Fallback gene expression data saved to {out_gene_data_file}")
    
    # Set gene_data to the fallback for downstream processing
    gene_data = gene_data_fallback


Handling Affymetrix Human Transcriptome Array 2.0 platform (GPL17586)...
Current gene_data shape: (70523, 57)

Creating a more robust gene mapping for Affymetrix HTA 2.0...
Created mapping with 70523 rows
Sample of mapping (first 5 rows):
ID: 2824546_st → Genes: ['PROBE_2824546']
ID: 2824549_st → Genes: ['PROBE_2824549']
ID: 2824551_st → Genes: ['PROBE_2824551']
ID: 2824554_st → Genes: ['PROBE_2824554']
ID: 2827992_st → Genes: ['PROBE_2827992']

Applying gene mapping with provisional probe-based identifiers...

Converted gene expression data: (0, 57) (genes × samples)

Fallback: Using original probe data with cleaned identifiers


Fallback gene expression data saved to ../../output/preprocess/Coronary_artery_disease/gene_data/GSE109048.csv
